In [ ]:
import math
import os
import tensorflow
import keras
from keras.utils import PyDataset
import numpy
tensorflow.config.list_physical_devices('GPU')

In [ ]:
NUM_SHOTS_ACCOUNT = 12
inputLayer = keras.Input(shape=(10 * NUM_SHOTS_ACCOUNT,))
hidden1 = keras.layers.Dense(64, activation="relu")(inputLayer)
hidden2 = keras.layers.Dense(32, activation="relu")(hidden1)
hidden3 = keras.layers.Dense(16, activation="relu")(hidden2)
outputLayer = keras.layers.Dense(1, activation="sigmoid")(hidden3)
model = keras.Model(inputs=inputLayer, outputs=outputLayer)
model.summary()

In [ ]:
# impletment dataset
class BowlingDataset(PyDataset):
    def __init__(self, fileName, batch_size, **kwargs):
        super().__init__(**kwargs)
        self.f = open(fileName, "rb")
        self.batch_size = batch_size
        self.length = int(os.stat(fileName).st_size/27)
        
    def __len__(self):
        return math.floor(self.length / self.batch_size)
    
    def __getitem__(self, idx):
        xArray = []
        yArray = []
        lowGame = idx * self.batch_size * 27
        self.f.seek(lowGame)
        for x in range(self.batch_size):
            gameData = self.f.read(27)
            tempArray = []
            for x in range(25):
                tempArray += [v for v in format(gameData[x], "08b")]
            finalScore = gameData[-1] * 256 + gameData[-2]
            xArray.append([float(v) for v in tempArray][:120])
            yArray.append(finalScore)
        return numpy.array(xArray), numpy.array(yArray)
            

In [ ]:
model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=0.0005)
)

In [ ]:
model.fit(BowlingDataset("ScoreDetailDataset.txt", 32, use_multiprocessing=True, workers=16, max_queue_size=32), shuffle=True, epochs=10)

In [ ]:
# since we just want one batch, initialize a bowlingdataset with a batch size equal to the number of entries
testDataset = BowlingDataset("ScoreDetailDatasetVSplit.txt", 1000)
error = model.test_on_batch(allData[0], allData[1])
print(error)

In [ ]:
name = str(int(NUM_SHOTS_ACCOUNT)) + ("-5" if NUM_SHOTS_ACCOUNT % 2 else "")
model.save("KerasModels/" + name + "Frames.keras")